In [36]:
import sys
import os
import random
import networkx as nx
sys.path.append(os.path.join(os.getcwd(), "src"))
from src.model import TrafficModel
from car import CarAgent


model = TrafficModel(50, num_intersections=5, num_borders=20, )

In [37]:
import pyoptinterface as poi
from pyoptinterface import highs

In [38]:
opt_model = highs.Model()

In [39]:
possible_lanes = ['intersection_4', 'intersection_7', 'intersection_1']
print(possible_lanes)

time = range(-1, 6)
print(time)

cars_at_light = {
                    0:{'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10},
                    1:{'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10},
                    2:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    3:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    4:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    5:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}
                 }
print(cars_at_light)

['intersection_4', 'intersection_7', 'intersection_1']
range(-1, 6)
{0: {'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10}, 1: {'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10}, 2: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 3: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 4: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 5: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}}


In [40]:
# Entscheidungsvariablen
lanes = opt_model.add_variables(time, possible_lanes, domain=poi.VariableDomain.Binary)

In [41]:
# Entscheidung für tick -1 ist bereits festgelegt (vorherige Entscheidung)
current_open_lane = 'intersection_4'

for lane in possible_lanes:
    if lane == current_open_lane:
        lanes[-1, lane] = 1
    else:
        lanes[-1, lane] = 0

In [42]:
# Restriktion: jede Ampel kann nur exakt eine open_lane haben
for tick in time[1:]:
    opt_model.add_linear_constraint(poi.quicksum(lanes[tick, lane] for lane in possible_lanes ), poi.Eq, 1)

# Restriktion: jede Ampel kann nur alle 5 Ticks die open_lane ändern (cooldown)
for lane in possible_lanes:
    opt_model.add_linear_constraint(poi.quicksum(((lanes[tick-1, lane] - lanes[tick, lane]) * (lanes[tick-1, lane] - lanes[tick, lane])) for tick in time[1:]), poi.Leq, 1.0)


In [43]:
# Zielfunktion
objective = poi.quicksum(poi.quicksum(lanes[tick, lane] * cars_at_light[tick][lane] for lane in possible_lanes) for tick in time[1:])
opt_model.set_objective(objective, poi.ObjectiveSense.Maximize)

In [44]:
opt_model.optimize()

In [45]:
for tick in time:
    for lane in possible_lanes:
        if opt_model.get_value(lanes[0, lane]) > 0.1:
            print(f"{tick}:{lane}")

-1:intersection_1
0:intersection_1
1:intersection_1
2:intersection_1
3:intersection_1
4:intersection_1
5:intersection_1


In [46]:
print(f"Lane: {opt_model.get_obj_value()}")

Lane: 60.0
